**#OR-Tools code**
- code 參考來源：**Google OR-tools Guides**
    - https://developers.google.com/optimization/scheduling/job_shop?hl=zh-tw

- 以下為code的運行過程 (以DMU44為例)

     1.  將 DMU 提供的txt檔用附件之 turn_into.cpp 轉換成以下的data變數。所有程式碼皆在vscode下執行，使用terminal進行快速轉換。
     ```
     turn_into.exe < dmu01.txt > out_dmu01.txt
     ```

In [1]:
import collections
from ortools.sat.python import cp_model

data = [  # task = (machine_id, processing_time).
        
[(0,142),(5,198),(1,135),(3,170),(2,12),(6,21),(4,134),(7,21),(9,33),(11,47),(10,130),(14,42),(13,96),(8,121),(12,166)],
[(2,153),(4,39),(3,129),(6,148),(1,5),(0,146),(5,191),(12,161),(14,74),(7,173),(10,113),(13,147),(9,172),(8,152),(11,125)],
[(2,64),(5,137),(3,93),(6,8),(1,185),(4,51),(0,111),(13,192),(10,165),(8,182),(7,90),(14,6),(9,110),(12,160),(11,18)],
[(6,153),(5,197),(0,191),(3,119),(1,191),(2,26),(4,121),(10,131),(7,179),(11,44),(14,163),(8,88),(12,185),(13,38),(9,77)],
[(2,105),(3,15),(4,125),(6,132),(5,130),(0,47),(1,164),(8,70),(9,48),(11,159),(14,97),(13,172),(7,194),(10,31),(12,82)],
[(1,62),(3,116),(4,182),(2,81),(6,176),(0,78),(5,10),(11,24),(13,157),(14,66),(8,13),(9,142),(10,189),(7,135),(12,98)],
[(3,28),(0,44),(5,190),(1,180),(2,129),(4,181),(6,33),(10,51),(11,60),(8,193),(12,74),(13,88),(14,103),(7,22),(9,93)],
[(5,177),(4,47),(6,72),(2,34),(0,152),(1,143),(3,63),(10,146),(13,183),(12,2),(11,166),(7,148),(14,10),(8,178),(9,149)],
[(6,157),(2,139),(0,53),(3,34),(5,42),(1,150),(4,114),(11,23),(13,171),(10,177),(8,147),(9,27),(14,109),(12,31),(7,76)],
[(3,63),(2,192),(4,166),(6,39),(0,50),(1,50),(5,179),(7,111),(10,35),(14,142),(8,136),(11,57),(12,3),(9,12),(13,55)],
[(3,183),(1,68),(0,41),(6,56),(2,165),(5,74),(4,53),(13,31),(8,80),(9,77),(14,168),(12,33),(10,97),(7,1),(11,112)],
[(1,157),(0,200),(6,113),(4,100),(5,2),(3,170),(2,41),(7,147),(10,48),(8,62),(9,168),(14,36),(12,77),(13,76),(11,121)],
[(2,133),(3,85),(5,89),(6,98),(0,56),(4,180),(1,15),(10,166),(13,160),(9,173),(8,194),(14,152),(7,136),(11,167),(12,171)],
[(0,49),(6,64),(5,174),(4,22),(3,32),(2,160),(1,35),(13,190),(12,137),(7,173),(9,2),(14,128),(10,5),(11,50),(8,39)],
[(2,115),(5,106),(1,19),(4,151),(3,76),(6,102),(0,187),(11,179),(7,184),(10,49),(9,68),(14,163),(12,137),(8,94),(13,54)],
[(0,127),(2,183),(1,26),(3,10),(6,73),(4,64),(5,5),(11,44),(10,2),(8,45),(12,46),(14,184),(13,97),(9,200),(7,86)],
[(1,84),(2,87),(3,179),(5,142),(6,12),(4,126),(0,169),(8,39),(9,168),(12,24),(13,142),(7,133),(14,147),(10,121),(11,179)],
[(6,98),(0,77),(5,41),(1,113),(2,97),(4,139),(3,17),(7,170),(9,116),(11,60),(12,165),(10,172),(14,77),(8,48),(13,196)],
[(4,35),(5,195),(1,3),(0,191),(6,55),(3,189),(2,5),(9,33),(11,131),(10,39),(14,158),(12,69),(8,100),(7,199),(13,169)],
[(6,131),(4,146),(1,175),(5,24),(2,50),(3,135),(0,78),(13,70),(7,152),(9,99),(14,152),(10,65),(12,124),(8,94),(11,100)]

    ]

2. 建立model
    ```
    model = cp_model.CpModel()
    ```

3. 定義變數、限制式、目標式
    - 這邊的定義是以下model改成code的
    >定義變數：t(i,j)，工件 i 在j th的工序中之開始時間      
    
    >限制式：   
    >優先順序：t(i,j)  + processingtime(i,j) < t(k,j)       
    >機台一次只能做一個工件：t(i,j) - t(i,k) > p(i,j)  or  t(i,k) - t(i,j) > p(j,j)          
    
    >目標式：minimize makespan

4. 丟入CP_solver並設定運行的限制時間
    ```
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 120.0
    status = solver.Solve(model)
    ```



In [2]:
def main():
    """Minimal jobshop problem."""
    # Data.
    jobs_data = data

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model.
    model = cp_model.CpModel()

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var,
                                                   end=end_var,
                                                   interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id +
                                1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 120.0
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solution:')
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(start=solver.Value(
                        all_tasks[job_id, task_id].start),
                                       job=job_id,
                                       index=task_id,
                                       duration=task[1]))

        # Create per machine output lines.
        output = ''
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_task_%i' % (assigned_task.job,
                                        assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-15s' % name

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                sol_line += '%-15s' % sol_tmp

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
        print(output)
    else:
        print('No solution found.')

    # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())

5.  運行後 output 結果

In [3]:
if __name__ == '__main__':
     
    main()

Solution:
Optimal Schedule Length: 3591.0
Machine 0: job_13_task_0  job_6_task_1   job_17_task_1  job_15_task_0  job_0_task_0   job_12_task_4  job_7_task_4   job_11_task_1  job_10_task_2  job_8_task_2   job_19_task_6  job_16_task_6  job_2_task_6   job_3_task_2   job_1_task_5   job_14_task_6  job_5_task_5   job_18_task_3  job_4_task_5   job_9_task_4   
           [0,49]         [49,93]        [98,175]       [175,302]      [302,444]      [463,519]      [519,671]      [671,871]      [871,912]      [912,965]      [969,1047]     [1047,1216]    [1216,1327]    [1327,1518]    [1546,1692]    [1692,1879]    [1879,1957]    [1957,2148]    [2211,2258]    [2480,2530]    
Machine 1: job_16_task_0  job_5_task_0   job_17_task_3  job_11_task_0  job_19_task_2  job_15_task_2  job_10_task_1  job_12_task_6  job_7_task_5   job_2_task_4   job_14_task_2  job_8_task_5   job_0_task_2   job_1_task_4   job_6_task_3   job_18_task_2  job_13_task_6  job_3_task_4   job_4_task_6   job_9_task_5   
           [0,84]     

- 因為output的要素過多，我們會再簡化成以下code，因為FlexSim Modle 只需知道順序即可

In [4]:
def main():
    """Minimal jobshop problem."""
    # Data.
    jobs_data = data

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model.
    model = cp_model.CpModel()

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var,
                                                   end=end_var,
                                                   interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id +
                                1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 120.0
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solution:')
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(start=solver.Value(
                        all_tasks[job_id, task_id].start),
                                       job=job_id,
                                       index=task_id,
                                       duration=task[1]))

        # Create per machine output lines.
        output = ''
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = '%i' % (assigned_task.job)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-5s' % name


            sol_line += '\n'

            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
        print(output)
    else:
        print('No solution found.')

    # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())
    

if __name__ == '__main__':
     
    main()

Solution:
Optimal Schedule Length: 3656.0
Machine 0: 13   6    17   11   7    0    4    10   15   18   12   8    2    1    19   3    14   9    16   5               
Machine 1: 11   5    17   16   10   2    7    13   18   4    19   14   1    12   8    15   6    9    3    0               
Machine 2: 4    2    12   7    17   1    16   13   14   8    11   10   19   15   9    18   5    6    3    0               
Machine 3: 6    9    4    10   2    5    17   13   12   1    7    11   16   8    19   14   15   18   3    0               
Machine 4: 18   7    4    17   13   1    11   19   5    2    12   14   10   8    15   9    16   6    3    0               
Machine 5: 7    17   2    13   4    18   11   12   14   3    19   8    10   1    6    16   0    9    15   5               
Machine 6: 17   19   7    13   4    2    11   3    8    12   10   1    18   15   14   9    16   5    6    0               
Machine 7: 17   11   13   7    19   2    1    10   14   9    12   3    0    4    16   18   5    1